In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\stat.py

C:\Users\keb7\AppData\Local\Temp\ipykernel_75988\1140549349.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sqlalchemy import create_engine
import pymssql
from datetime import datetime


codes = ['F','G','H','J','K','M','N','Q','U','V','X','Z']
codeToMonth = dict(zip(codes,range(1,13)))

def expiry(code) :
    year = code[-4:]
    month = codeToMonth[code[-5]]
    month = str(month) if month>=10 else '0'+str(month)
    return datetime.strptime(year+month,'%Y%m')

server = 'fs.rice.edu'
database = 'futures'
username = 'futures'
password = 'RCGCMO'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

df = pd.read_sql(" select * from data where code like 'CME_GC%' and date='2022-01-24' ", conn)    
df['expiry'] = df.code.apply(expiry)
df = df.sort_values(by='expiry').set_index('expiry')
trace = go.Scatter(
    x=df.index.to_list(),
    y=df.settle,
    mode="lines",
    hovertemplate="expiration = %{x}<br>futures price = $%{y:,.2f}<extra></extra>"
)

fig = go.Figure()
fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Expiration",
    yaxis_title="Futures Price",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    height=600,  
    width=1000,
    font_size=16
)
fig.show()

In [3]:
df = pd.read_sql(" select * from data where code like 'CME_ES%' and date='2022-01-24' ", conn)    
df['expiry'] = df.code.apply(expiry)
df = df.sort_values(by='expiry').set_index('expiry')
trace = go.Scatter(
    x=df.index.to_list(),
    y=df.settle,
    mode="lines",
    hovertemplate="expiration = %{x}<br>futures price = $%{y:,.2f}<extra></extra>"
)

fig = go.Figure()
fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Expiration",
    yaxis_title="Futures Price",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    height=600,  
    width=1000,
    font_size=16
)
fig.show()